In [ ]:
from bs4 import BeautifulSoup
import requests
import requests.exceptions
from urllib.parse import urlsplit
from urllib.parse import urlparse
from collections import deque
import re
import lxml
import time
research_articles = []

url = "https://badgerherald.com/"
# a queue of urls to be crawled
new_urls = deque([url])

# a set of urls that we have already been processed 
processed_urls = set()
# a set of domains inside the target website
local_urls = set()
# a set of domains outside the target website
foreign_urls = set()
# a set of broken urls
broken_urls = set()

# process urls one by one until we exhaust the queue
x = 0

In [ ]:
while len(new_urls):
    # move next url from the queue to the set of processed urls
    url = new_urls.popleft()
    processed_urls.add(url)
    # get url's content
    x = x+1
    if x%50 == 0:
        print("processed " + str(x) + " urls")
        print(len(research_articles))
        time.sleep(8)
    try:
        response = requests.get(url)
    except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError, requests.exceptions.InvalidURL, requests.exceptions.InvalidSchema):
        # add broken urls to it's own set, then continue
        broken_urls.add(url)
        continue
    time.sleep(2)
    # extract base url to resolve relative links
    parts = urlsplit(url)
    base = "{0.netloc}".format(parts)
    strip_base = base.replace("www.", "")
    base_url = "{0.scheme}://{0.netloc}".format(parts)
    path = url[:url.rfind('/')+1] if '/' in parts.path else url

    # create a beutiful soup for the html document
    soup = BeautifulSoup(response.text, "lxml")
    
    ###########origingla code#############
    key_words = ["sexual assault", "sexual misconduct", "rape", "sexual harassment", "sexual abuse"]
    for phrase in key_words:
        for element in soup.find_all("p"):
            if phrase in element.text and (url not in research_articles):
                research_articles.append(url)
        for element in soup.find_all("h2"):
            if phrase in element.text and (url not in research_articles):
                research_articles.append(url)
        for element in soup.find_all("h1"):
            if phrase in element.text and (url not in research_articles):
                research_articles.append(url)

    ######################################
    for link in soup.find_all('a'):
        # extract link url from the anchor
        anchor = link.attrs["href"] if "href" in link.attrs else ''
        if anchor.startswith('/'):
            local_link = base_url + anchor
            local_urls.add(local_link)
        elif strip_base in anchor:
            local_urls.add(anchor)
        elif not anchor.startswith('http'):
            local_link = path + anchor
            local_urls.add(local_link)
        else:
            foreign_urls.add(anchor)

        for i in local_urls:
            if not i in new_urls and not i in processed_urls:
                if "banter" not in i:
                    new_urls.append(i)

print("finished: " + str(len(processed_urls)))

processed 5900 urls
429
processed 5950 urls
430
processed 6000 urls
430
processed 6050 urls
430
processed 6100 urls
430
processed 6150 urls
432
processed 6200 urls
432
processed 6250 urls
434
processed 6300 urls
434
processed 6350 urls
435


In [5]:
import json
with open('Badger_Herald_links2.json', 'w') as outfile:
    json.dump(research_articles, outfile)

In [ ]:
processed 4950 urls
400